In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 500

In [13]:
df = pd.read_csv("./output/resampling_upscale_0.0833333mins_20len_FORMATED.csv",delimiter="\t", names=["time","traj_id","x","y"])

In [14]:
df.head()

,time,traj_id,x,y
0,0,1,8.560,10.440
1,1,1,7.510,10.505
2,2,1,6.460,10.570
3,3,1,5.790,10.560
4,4,1,4.865,10.545


In [22]:
bg_store_img_metadata = {'path':"./data/german_1/storePlan_fix.jpg",'image_crop':[0, 45.6, 0, 26.7], "x_lim": (-1, 44.56), "y_lim" : (26.40, -1) }
def putBgStoreImage(ax, plotFunc = None):
    bg_img = mpimg.imread(bg_store_img_metadata["path"])
    
    ax.xaxis.tick_top()

    ax.imshow(bg_img, extent=bg_store_img_metadata["image_crop"], origin="lower", aspect="equal")

    ax.set_xlim(bg_store_img_metadata["x_lim"])
    ax.set_ylim(bg_store_img_metadata["y_lim"])

In [32]:
def trajectoryAnimation(traj_data):
    traj_data = traj_data.sort_values(by="time")
    
    fig, ax = plt.subplots(figsize=(12,6))
    putBgStoreImage(ax)
     
    square, = ax.plot(traj_data.x.iloc[0], traj_data.y.iloc[0], color = 'blue', marker = 's', markersize=2 )
    line, = ax.plot(traj_data.x.iloc[0:1], traj_data.y.iloc[0:1], color = 'grey' )
    dot, = ax.plot(traj_data.x.iloc[0], traj_data.y.iloc[0], color = 'black', marker = 'o' )

    def update_frame(t):
        square.set_data([traj_data.x.iloc[0:t+1]], [traj_data.y.iloc[0:t+1]])
        line.set_data([traj_data.x.iloc[0:t+1]], [traj_data.y.iloc[0:t+1]])
        dot.set_data([traj_data.x.iloc[t]], [traj_data.y.iloc[t]])
        ax.set_title(f'Relationship between x and y at step {t}', fontsize=14)
        return square, line, dot
    
    time = np.arange(len(traj_data.x))
    anim = animation.FuncAnimation(fig, update_frame, frames=time, interval=100)
    
    anim.save(f'./output/Gifs/{traj_data.traj_id.iloc[0]}_{traj_data.traj_id.iloc[0]}.gif', writer='pillow')
    return anim

In [ ]:
trajectoryAnimation(df[df.traj_id == 1])

*Data Filtering and Smoothing*